In [2]:
import pandas as pd
import numpy as np
import matplotlib as plt

In [22]:
#import
df_matches = pd.read_csv("WorldCupMatches.csv")
df_players = pd.read_csv("WorldCupPlayers.csv")
df_world_cups = pd.read_csv("WorldCups.csv")

In [23]:
df_player.describe().T

,count,mean,std,min,25%,50%,75%,max
RoundID,37784.0,1.105647e+07,2.770144e+07,201.0,263.0,337.0,255931.0,97410600.0
MatchID,37784.0,6.362233e+07,1.123916e+08,25.0,1199.0,2216.0,97410003.0,300186515.0
Shirt Number,37784.0,1.072602e+01,6.960138e+00,0.0,5.0,11.0,17.0,23.0
